# Quantum State Tomography

Below is a simple code capable of performing quantum state tomography on a random quantum state. First, the quantum state is generated by sending the $|0\rangle$ state through Z, Y, and Z rotations parametrized by random $\theta_1, \theta_2,$ and $ \theta_3$, respectively.  Then, we perform measurements in the Pauli matrices' eigenvector basis so that we can compute the Stokes parameters. Remember that the Stokes parameters are defined as $S_0 = 1$ and $S_i = P_{|\psi_i\rangle} - P_{|\psi_i^\bot\rangle}$ for $i = 1, 2, 3\,$, where $|\psi_i\rangle$ is the eigenvector of eigenvalue 1 of the $i^{th}$ Pauli matrix and $P_{|\psi_i\rangle}$ the probability of measuring said state. We can finally compute the density matrix of the final state as $\rho = \frac{1}{2}\sum_{i = 0}^3 S_i \sigma_i\,$.

In [85]:
import numpy as np
from qibo import Circuit, gates

# Definition of parameters
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 3)
S = np.ones(4)
Pauli = np.array([np.eye(2),[[0, 1], [1, 0]],[[0, -1j], [1j, 0]],[[1, 0], [0, -1]]], dtype = np.matrix)

# We first run the circuit without performing any measurements so that we can compare the real density matrix with our approximation
c0 = Circuit(1, density_matrix = True)
c0.add(gates.RZ(0, theta[0]))
c0.add(gates.RY(0, theta[1]))
c0.add(gates.RZ(0, theta[2]))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

# We perform measurements in each of the Pauli matrices eigenvector basis
for i in range(1, 4):
    c = Circuit(1)
    c.add(gates.RZ(0, theta[0]))
    c.add(gates.RY(0, theta[1]))
    c.add(gates.RZ(0, theta[2]))

    # Change of basis
    if(i == 1): c.add(gates.H(0))
    elif(i == 2):
        c.add(gates.S(0).dagger)
        c.add(gates.H(0))

    # Measurement
    c.add(gates.M(0))
    result = c(nshots = 1000000)

    freq = result.frequencies()
    S[i] = (freq['0'] - freq['1'])/(freq['0']+freq['1'])
    print()
    print("Measurement of S_", i, ":",  sep = '')
    print(c.draw())
    print(freq)

# Compute of density matrix
rho = 0.5*np.eye(2)
for i in range(1, 4):
    rho = rho + 0.5*S[i]*Pauli[i]

print()
print("Tomographic approximation of the density matrix")
print(rho)



Real density matrix:
[[0.96239578+0.j         0.07626519-0.17428071j]
 [0.07626519+0.17428071j 0.03760422+0.j        ]]
q0: ─RZ─RY─RZ─

Measurement of S_1:
q0: ─RZ─RY─RZ─H─M─
Counter({'0': 575906, '1': 424094})

Measurement of S_2:
q0: ─RZ─RY─RZ─S─H─M─
Counter({'1': 673505, '0': 326495})

Measurement of S_3:
q0: ─RZ─RY─RZ─M─
Counter({'0': 962833, '1': 37167})

Tomographic approximation of the density matrix
[[0.962833 (0.075906+0.173505j)]
 [(0.075906-0.173505j) 0.037167000000000006]]


For n-qubit states, tomography becomes much more complicated. Instead of 3 Stokes parameters, we need $4^n - 1$. First, a random state is generated by sending the $|0\rangle$ state through a contraption parametrized by random $\theta_i$. Then, we loop over all possible Stokes parameters; to compute $S_{i_1, \dots, i_n}$, we will make projective measurements in the basis $\{|\psi_{i_1, j_1}\rangle\otimes\cdots\otimes|\psi_{i_n, j_n}\rangle\}_{j_1, \dots, j_n}$, where $\{|\psi_{i, j}\rangle\}_j$ are the eigenvectors of the $i^{th}$ Pauli matrix. Remember that $ S_{i_1, \dots, i_n} = \bigotimes_{k = 1}^n \left(P_{|\phi_{i_k}\rangle} \pm P_{|\phi_{i_k}^\bot \rangle }\right) = \sum_{k_1, \dots, k_n} \pm P_{|\phi_{i_{k_1}}\rangle\cdots|\phi_{i_{k_n}}\rangle}\,$. On the tensor product, the + sign is used for 0 indeces and the - sign for 1, 2 or 3 indeces. Thus, on the summatory, the sign will be -1 to the power of the number of $\bot$ terms that don't correspond to a 0 subindex. 

In [94]:
import numpy as np
import qibo
from qibo import Circuit, gates
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z

#Returns the number such that its binary representation in n bits is the inverse of l's binary representation 
def invert_binary(l, n):
    s = 0
    for i in range(n):
        s += (l%2)*(2**(n - 1 - i))
        l //= 2
    return s

n = 5 #number of qubits
m = 10 #number of rng layers
shots = 1000000 #number of shots
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 2*n*m)
rho = 0
c0 = Circuit(n, density_matrix = True)
Pauli = np.array([I, X, Y, Z], dtype = qibo.symbols.Symbol)


#Generates the state and prints its density matrix
for i in range(m): 
    for j in range(n):
        c0.add(gates.RY(j, theta[2*n*i+2*j]))
        c0.add(gates.RZ(j, theta[2*n*i+2*j+1]))
        c0.add(gates.CNOT(j, (j+1)%n))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

#Loop over all possible n-quatrit quaternary codes. 
for k in range(0, 4**n):
    c = Circuit(n)

    #Generates state
    for i in range(m): 
        for j in range(n):
            c.add(gates.RY(j, theta[2*n*i+2*j]))
            c.add(gates.RZ(j, theta[2*n*i+2*j+1]))
            c.add(gates.CNOT(j, (j+1)%n))

    #Change of basis
    for l in range(n):
        if(((k//(4**l))%4) == 1) :
            c.add(gates.H(l))
        elif(((k//(4**l))%4) == 2) :
            c.add(gates.S(l).dagger())
            c.add(gates.H(l))
    
    #Measurement
    c.add(gates.M(*range(n)))
    result = c(nshots = shots)
    freq = result.frequencies(binary = False)

    #Computes the k-th Stokes parameter
    S = 0
    for l in range(2**n):
        sign = 1
        for j in range(n):
            if((l//(2**j))%2 == 1 and (k//(4**j))%4 != 0):
                sign *= -1

        S += sign * freq[invert_binary(l, n)]
    S /= shots

    #Adds the k-th Pauli string term to rho
    aux = k
    P = Pauli[aux%4](0)
    aux //= 4
    for i in range(1, n):
        P = P*Pauli[aux%4](i)
        aux //= 4
    rho = rho + (1/(2**n))*S

#Print results
rho = SymbolicHamiltonian(rho).matrix
print(rho)
print('Fidelity = ', (rho@c0().state()).trace())

Real density matrix:
[[ 0.02191131+5.20417043e-18j -0.01044622-9.57607908e-03j
  -0.00861944+3.52164657e-02j ... -0.01543709-2.37225603e-02j
  -0.0338168 +1.61908712e-02j  0.02865656-3.76572374e-02j]
 [-0.01044622+9.57607908e-03j  0.00916535+3.33934269e-17j
  -0.01128162-2.05564745e-02j ...  0.0177273 +4.56313248e-03j
   0.00904613-2.24982255e-02j  0.00279563+3.04770953e-02j]
 [-0.00861944-3.52164657e-02j -0.01128162+2.05564745e-02j
   0.05999157+3.46944695e-18j ... -0.03205493+3.41428683e-02j
   0.03932521+4.79821389e-02j -0.07179662-3.12440550e-02j]
 ...
 [-0.01543709+2.37225603e-02j  0.0177273 -4.56313248e-03j
  -0.03205493-3.41428683e-02j ...  0.03655937-1.73472348e-17j
   0.00629557-4.80190710e-02j  0.02058079+5.75558914e-02j]
 [-0.0338168 -1.61908712e-02j  0.00904613+2.24982255e-02j
   0.03932521-4.79821389e-02j ...  0.00629557+4.80190710e-02j
   0.06415498-1.73472348e-17j -0.07205302+3.69431259e-02j]
 [ 0.02865656+3.76572374e-02j  0.00279563-3.04770953e-02j
  -0.07179662+3.12440

[Qibo 0.2.9|WARNING|2024-07-18 16:14:05]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[[ 0.02202525+0.j         -0.010615  -0.00959306j -0.00865488+0.03503725j
  ... -0.01529063-0.02380125j -0.03366213+0.01614819j
   0.02861581-0.03759725j]
 [-0.010615  +0.00959306j  0.00933687+0.j         -0.01124537-0.02032256j
  ...  0.01757694+0.00453212j  0.00914156-0.02249975j
   0.002577  +0.03062731j]
 [-0.00865488-0.03503725j -0.01124537+0.02032256j  0.06018075+0.j
  ... -0.03207606+0.034057j    0.03923569+0.047917j
  -0.07188737-0.03137175j]
 ...
 [-0.01529063+0.02380125j  0.01757694-0.00453212j -0.03207606-0.034057j
  ...  0.03666638+0.j          0.00634188-0.04796256j
   0.02043938+0.05769337j]
 [-0.03366213-0.01614819j  0.00914156+0.02249975j  0.03923569-0.047917j
  ...  0.00634188+0.04796256j  0.06410725+0.j
  -0.072056  +0.03697006j]
 [ 0.02861581+0.03759725j  0.002577  -0.03062731j -0.07188737+0.03137175j
  ...  0.02043938-0.05769337j -0.072056  -0.03697006j
   0.10222663+0.j        ]]
Fidelity =  (0.9998186285615753-2.969358699894675e-17j)


This code is much faster than the previous one. Instead of measuring in a new basis for each of the $4^n - 1$ Stokes parameters, it only uses $3^n$ settings, corresponding to all the possible Stoke parameters subindeces that don't include 0s. The Stokes parameters whose subindeces include 0s can be computed from the same set of probabilities as those that don't, but share the same non-zero terms. Thus, after each set of probabilities is obtained, the code loops over those subindeces that can be obtained by substituting some of the quatrits by 0. An array that keeps track of the n-quatrit codes already visited helps avoid duplicates. 

In [109]:
import numpy as np
import qibo
from qibo import Circuit, gates
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z

#Returns the number such that its binary representation in n bits is the inverse of l's binary representation 
def invert_binary(l, n):
    s = 0
    for i in range(n):
        s += (l%2)*(2**(n - 1 - i))
        l //= 2
    return s

#Interpreting n as a code in tertiary and m as a code in binary, returns a code in quaternary
#such that the i-th character is 0 if m's i-th character is, and n's i-th character + 1 otherwise
def and_3_2(n, m):
    if(m == 0): return 0
    return 4*and_3_2(n//3, m//2) + (n%3 + 1)*(m%2)

#Returns the number of ones in the binary representation of n
def number_of_ones(n):
    if(n < 2): return n
    return number_of_ones(n//2) + n%2

n = 5 #number of qubits
m = 1 #number of rng layers
shots = 1000000 #number of shots
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 2*n*m)
rho = 0
c0 = Circuit(n, density_matrix = True)
Pauli = np.array([I, X, Y, Z], dtype = qibo.symbols.Symbol)
visited = np.zeros(4**n, dtype = bool)

#Generates the state and prints its density matrix
for i in range(m): 
    for j in range(n):
        c0.add(gates.RY(j, theta[2*n*i+2*j]))
        c0.add(gates.RZ(j, theta[2*n*i+2*j+1]))
        c0.add(gates.CNOT(j, (j+1)%n))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

#Loop over all the posible n-trit tertiary codes.
for k in range(0, 3**n):
    c = Circuit(n)
    
    #Generates state
    for i in range(m): 
        for j in range(n):
            c.add(gates.RY(j, theta[2*n*i+2*j]))
            c.add(gates.RZ(j, theta[2*n*i+2*j+1]))
            c.add(gates.CNOT(j, (j+1)%n))

    #Change of basis
    for l in range(n):
        if(((k//(3**l))%3) == 0) :
            c.add(gates.H(l))
        elif(((k//(3**l))%3) == 1) :
            c.add(gates.S(l).dagger())
            c.add(gates.H(l))
    
    #Measurementt 
    c.add(gates.M(*range(n)))
    result = c(nshots = shots)
    freq = result.frequencies(binary = False)

    #Computes as many of the 4^n - 1 Stokes parameters as possible and sums the corresponding
    #term to rho, but only if the parameter hasn't already been calculated
    for i in range(2**n):
        s = and_3_2(k, i)
        if(not visited[s]):
            visited[s] = True
            S = 0
            for l in range(2**n):
                sign = (-1)**(number_of_ones(i&l)%2)
                S += sign * freq[invert_binary(l, n)]
            S /= shots
            
            P = Pauli[s%4](0)
            s //= 4
            for j in range(1, n):
                P = P*Pauli[s%4](j)
                s //= 4
            rho = rho + (1/(2**n))*S*P


#Print results
rho = SymbolicHamiltonian(rho).matrix
print(rho)
print('Fidelity = ', (rho@c0().state()).trace())

Real density matrix:
[[ 0.07150702-6.93889390e-18j  0.06149862-6.86093388e-02j
   0.00103245+1.50209959e-02j ... -0.01967122-1.69975912e-02j
  -0.02877099+4.49404729e-02j  0.07412983-9.87394927e-02j]
 [ 0.06149862+6.86093388e-02j  0.11872011-6.93889390e-18j
  -0.01352436+1.39092065e-02j ... -0.00060916-3.34926263e-02j
  -0.06786344+1.10453285e-02j  0.1584926 -1.37936593e-02j]
 [ 0.00103245-1.50209959e-02j -0.01352436-1.39092065e-02j
   0.00317027+2.16840434e-19j ... -0.00385459+3.88678281e-03j
   0.00902493+6.69259447e-03j -0.01967122-1.69975912e-02j]
 ...
 [-0.01967122+1.69975912e-02j -0.00060916+3.34926263e-02j
  -0.00385459-3.88678281e-03j ...  0.00945187-8.67361738e-19j
  -0.00276783-1.92019120e-02j  0.00307815+4.47837860e-02j]
 [-0.02877099-4.49404729e-02j -0.06786344-1.10453285e-02j
   0.00902493-6.69259447e-03j ... -0.00276783+1.92019120e-02j
   0.03982009+0.00000000e+00j -0.09188172-6.86082348e-03j]
 [ 0.07412983+9.87394927e-02j  0.1584926 +1.37936593e-02j
  -0.01967122+1.69975

[Qibo 0.2.9|WARNING|2024-07-18 16:43:31]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[[ 0.0712535 +0.j          0.06145687-0.06864556j  0.00094181+0.01497669j
  ... -0.01964388-0.01682194j -0.02877975+0.04519437j
   0.07393056-0.09878069j]
 [ 0.06145687+0.06864556j  0.11887237+0.j         -0.0134945 +0.01382812j
  ... -0.00086062-0.03362625j -0.06785131+0.01130044j
   0.1584975 -0.013737j  ]
 [ 0.00094181-0.01497669j -0.0134945 -0.01382812j  0.00311912+0.j
  ... -0.00371669+0.00388081j  0.00904512+0.006635j
  -0.0198425 -0.01709131j]
 ...
 [-0.01964388+0.01682194j -0.00086062+0.03362625j -0.00371669-0.00388081j
  ...  0.00927762+0.j         -0.00278138-0.01908625j
   0.00322644+0.04476219j]
 [-0.02877975-0.04519437j -0.06785131-0.01130044j  0.00904512-0.006635j
  ... -0.00278138+0.01908625j  0.03956238+0.j
  -0.0918985 -0.00689519j]
 [ 0.07393056+0.09878069j  0.1584975 +0.013737j   -0.0198425 +0.01709131j
  ...  0.00322644-0.04476219j -0.0918985 +0.00689519j
   0.21299525+0.j        ]]
Fidelity =  (1.0002427241741012-3.957337929572091e-18j)
